In [19]:
import os
import sys
from glob import glob
from tqdm import tqdm
import math
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
import torch
from torch.nn import functional as F
from torch import nn
from torch.utils.data import DataLoader
sys.path.insert(0, '../')
from utils import load_json, save_pickle
from model import VanillaEfficientNet, load_model
from dataset import TrainDataset, EvalDataset
from transform_settings import configure_transform

- CutMix
- Data Imbalance

In [2]:
INFO = '../preprocessed/info.pkl'
META = '../preprocessed/metadata.json'
TASK = 'age'
train_transform = configure_transform('train', 'base')
valid_transform = configure_transform('valid', 'base')
infer_transform = configure_transform('eval', 'base')
metadate = load_json('../preprocessed/metadata.json')

In [3]:
config_age = ('../preprocessed/valid', configure_transform('valid', 'base'), 'age', META)
config_ageg = ('../preprocessed/valid', configure_transform('valid', 'base'), 'ageg', META)

validset_age = TrainDataset(*config_age)
validset_ageg = TrainDataset(*config_ageg)

validloader_age = DataLoader(validset_age, batch_size=32, shuffle=False, drop_last=False)
validloader_ageg = DataLoader(validset_ageg, batch_size=32, shuffle=False, drop_last=False)

In [5]:
AGE = '../saved_models/VanillaEfficientNet_taskage_epoch03_lr0.005_transformbase_optimadam_loss25.2501_eval5.0249_seed42.pth'
AGEG = '../saved_models/VanillaEfficientNet_taskageg_epoch04_lr0.005_transformbase_optimadam_loss0.0002_eval0.9118_seed42.pth'


model_age = load_model('VanillaEfficientNet', 'age', AGE)
model_ageg = load_model('VanillaEfficientNet', 'ageg', AGEG)

model_age.cuda()
model_ageg.cuda()

model_age.eval()
model_ageg.eval()

print('Cuda, Eval')

Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights from ../saved_models/VanillaEfficientNet_taskage_epoch03_lr0.005_transformbase_optimadam_loss25.2501_eval5.0249_seed42.pth
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights from ../saved_models/VanillaEfficientNet_taskageg_epoch04_lr0.005_transformbase_optimadam_loss0.0002_eval0.9118_seed42.pth
Cuda, Eval


In [6]:
@np.vectorize
def age2ageg(age):
    if age < 30:
        return 0
    elif age >= 30 and age < 60:
        return 1
    else:
        return 2

Predict by Regression

In [7]:
pred_list = []
gt_list = []

for imgs, labels in tqdm(validloader_age):
    imgs = imgs.cuda()
    labels = labels.data.numpy()
    output = model_age(imgs)

    preds_ageg = age2ageg(output.data.cpu().numpy().flatten())
    labels_ageg = age2ageg(labels)

    pred_list.append(preds_ageg)
    gt_list.append(labels_ageg)

100%|██████████| 89/89 [00:16<00:00,  5.49it/s]


In [21]:
# evaluation for clf(ageg)
pred_list = []
true_list = []

# evaluation for reg(age)
mse_raw = 0
rmse_raw = 0
num_samples = 0
criterion = nn.MSELoss()

i = 0
with torch.no_grad():
    model_age.eval()
    for imgs, labels in tqdm(validloader_age, desc="Valid"):
        imgs = imgs.cuda()
        output = model_age(imgs)

        # regression(age)
        labels_reg = labels.float().cuda()
        mse_raw += criterion(output, labels_reg.unsqueeze(1)).item() * len(
            labels_reg
        )
        rmse_raw += F.mse_loss(output, labels_reg.unsqueeze(1)).item() * len(
            labels_reg
        )
        num_samples += len(labels_reg)

        # classification(ageg)
        labels_clf = age2ageg(labels.data.numpy())
        preds_clf = age2ageg(output.data.cpu().numpy().flatten())
        pred_list.append(preds_clf)
        true_list.append(labels_clf)

        train_rmse = math.sqrt(rmse_raw / num_samples)
        train_mse = mse_raw / num_samples

        pred_arr = np.hstack(pred_list)
        true_arr = np.hstack(true_list)

        train_acc = (true_arr == pred_arr) / len(true_arr)
        train_f1 = f1_score(y_true=true_arr, y_pred=pred_arr, average="macro")
        if i == 3:
            break
        i += 1

Valid:   3%|▎         | 3/89 [00:00<00:21,  4.06it/s]


In [24]:
pred_arr

array([0, 0, 0, 0, 1, 2, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0])

In [22]:
train_acc

array([0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.       ,
       0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.       ,
       0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125,
       0.0078125, 0.0078125, 0.0078125, 0.       , 0.0078125, 0.0078125,
       0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125,
       0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125,
       0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.       , 0.0078125,
       0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.       ,
       0.0078125, 0.       , 0.0078125, 0.0078125, 0.0078125, 0.0078125,
       0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125,
       0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.       ,
       0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125,
       0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125, 0.0078125,
       0.0078125, 0.0078125, 0.0078125, 0.       , 

In [10]:
pred_arr

array([0, 0, 0, ..., 1, 1, 0])

In [40]:
num_corrects = (pred_arr == gt_arr).sum()
num_samples = pred_arr.shape[0]
num_corrects / num_samples

0.816931216931217

In [49]:
f1_score(y_true=gt_arr, y_pred=pred_arr, average='macro')

0.6319533935359928

Predict by Classification

In [18]:
pred_list = []
gt_list = []

for imgs, labels in tqdm(validloader_ageg):
    imgs = imgs.cuda()

    output = model_ageg(imgs)
    _, preds = torch.max(output, 1)

    pred_list.append(preds.data.cpu().numpy())
    gt_list.append(labels.numpy())

100%|██████████| 89/89 [00:15<00:00,  5.75it/s]


In [20]:
pred_arr = np.hstack(pred_list)
gt_arr = np.hstack(gt_list)

In [27]:
f1_score(gt_arr, np.array([0.5 for _ in range(gt_arr.shape[0])]), average='macro')

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

array([0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5])

In [43]:
pred_arr_ageg = np.hstack(pred_list)
gt_arr_ageg = np.hstack(gt_list)

In [45]:
(gt_arr_ageg == gt_arr).sum() == gt_arr.shape[0]

True

In [46]:
num_corrects = (pred_arr_ageg == gt_arr_ageg).sum()
num_samples = pred_arr_ageg.shape[0]
num_corrects / num_samples

0.6338624338624339

In [48]:
f1_score(y_true=gt_arr_ageg, y_pred=pred_arr_ageg, average='macro')

0.5165707653767123